# **Data Preprocessing**

In [1]:
# importing required libraries
import random
import math
import numpy as np
import pandas as pd

import pickle # saving and loading trained model
from os import path

# importing required libraries for normalizing data
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# importing library for plotting
import matplotlib.pyplot as plt

# importing library for support vector machine classifier
from sklearn.svm import SVC
# importing library for K-neares-neighbor classifier
from sklearn.neighbors import KNeighborsClassifier
# importing library for Linear Discriminant Analysis Model
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# importing library for Quadratic Discriminant Analysis Model
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn import metrics
from sklearn.metrics import accuracy_score # for calculating accuracy of model
from sklearn.model_selection import train_test_split # for splitting the dataset for training and testing
from sklearn.metrics import classification_report # for generating a classification report of model

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc

from keras.layers import Dense # importing dense layer
from keras.models import Sequential #importing Sequential layer
from keras.models import model_from_json # saving and loading trained model

from keras.layers import LSTM
from keras.layers import Input
from keras.models import Model

# representation of model layers
from keras.utils.vis_utils import plot_model

2022-12-16 19:19:19.058743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 19:19:19.058867: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# dataset doesn't have column names, so we have to provide it
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty_level"]

In [ ]:
# upload dataset to google colab
from google.colab import files
from io import BytesIO

uploaded = files.upload()

fn = list(uploaded.keys())[0]

data = pd.read_csv(BytesIO(uploaded[fn]), header=None, names=col_names)

In [3]:
# importing dataset
#data = pd.read_csv('/content/drive/MyDrive/NSL-KDD/KDDTrain+.txt',header=None, names=col_names)
data = pd.read_csv('datasets/KDDTrain+.txt',header=None, names=col_names)

In [4]:
# print dataset
data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,difficulty_level
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,...,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,...,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,...,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0,tcp,private,S0,0,0,0,0,0,0,...,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20
125969,8,udp,private,SF,105,145,0,0,0,0,...,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal,21
125970,0,tcp,smtp,SF,2231,384,0,0,0,0,...,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal,18
125971,0,tcp,klogin,S0,0,0,0,0,0,0,...,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,20


In [5]:
# remove attribute 'difficulty_level'
data.drop(['difficulty_level'],axis=1,inplace=True)
data.shape

(125973, 42)

In [6]:
# descriptive statistics of dataset
data.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,125973.0,287.144650,2.604515e+03,0.0,0.00,0.00,0.00,4.290800e+04
src_bytes,125973.0,45566.743000,5.870331e+06,0.0,0.00,44.00,276.00,1.379964e+09
dst_bytes,125973.0,19779.114421,4.021269e+06,0.0,0.00,0.00,516.00,1.309937e+09
land,125973.0,0.000198,1.408607e-02,0.0,0.00,0.00,0.00,1.000000e+00
wrong_fragment,125973.0,0.022687,2.535300e-01,0.0,0.00,0.00,0.00,3.000000e+00
urgent,125973.0,0.000111,1.436603e-02,0.0,0.00,0.00,0.00,3.000000e+00
hot,125973.0,0.204409,2.149968e+00,0.0,0.00,0.00,0.00,7.700000e+01
num_failed_logins,125973.0,0.001222,4.523914e-02,0.0,0.00,0.00,0.00,5.000000e+00
logged_in,125973.0,0.395736,4.890101e-01,0.0,0.00,0.00,1.00,1.000000e+00
num_compromised,125973.0,0.279250,2.394204e+01,0.0,0.00,0.00,0.00,7.479000e+03


In [7]:
# number of attack labels 
data['label'].value_counts()

normal             67343
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: label, dtype: int64

In [8]:
# changing attack labels to their respective attack class
def change_label(df):
  df.label.replace(['apache2','back','land','neptune','mailbomb','pod','processtable','smurf','teardrop','udpstorm','worm'],'Dos',inplace=True)
  df.label.replace(['ftp_write','guess_passwd','httptunnel','imap','multihop','named','phf','sendmail',
       'snmpgetattack','snmpguess','spy','warezclient','warezmaster','xlock','xsnoop'],'R2L',inplace=True)
  df.label.replace(['ipsweep','mscan','nmap','portsweep','saint','satan'],'Probe',inplace=True)
  df.label.replace(['buffer_overflow','loadmodule','perl','ps','rootkit','sqlattack','xterm'],'U2R',inplace=True)

In [9]:
# calling change_label() function
change_label(data)

In [10]:
# distribution of attack classes
data.label.value_counts()

normal    67343
Dos       45927
Probe     11656
R2L         995
U2R          52
Name: label, dtype: int64

# **Data Normalization**

In [11]:
# selecting numeric attributes columns from data
numeric_col = data.select_dtypes(include='number').columns

In [12]:
# using standard scaler for normalizing
std_scaler = StandardScaler()
def normalization(df,col):
  for i in col:
    arr = df[i]
    arr = np.array(arr)
    df[i] = std_scaler.fit_transform(arr.reshape(len(arr),1))
  return df

In [13]:
# data before normalization
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos
3,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal


In [14]:
# calling the normalization() function
data = normalization(data.copy(),numeric_col)

In [15]:
# data after normalization
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.818890,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.035688,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.809857,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,Dos
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.258754,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal


# **One-hot-encoding**

In [16]:
# selecting categorical data attributes
cat_col = ['protocol_type','service','flag']

In [17]:
# creating a dataframe with only categorical attributes
categorical = data[cat_col]
categorical.head()

,protocol_type,service,flag
0,tcp,ftp_data,SF
1,udp,other,SF
2,tcp,private,S0
3,tcp,http,SF
4,tcp,http,SF


In [18]:
# one-hot-encoding categorical attributes using pandas.get_dummies() function
categorical = pd.get_dummies(categorical,columns=cat_col)
categorical.head()

,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,service_courier,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


# **Binary Classification**

In [19]:
# changing attack labels into two categories 'normal' and 'abnormal'
bin_label = pd.DataFrame(data.label.map(lambda x:'normal' if x=='normal' else 'abnormal'))

In [20]:
# creating a dataframe with binary labels (normal,abnormal)
bin_data = data.copy()
bin_data['label'] = bin_label

In [21]:
# label encoding (0,1) binary labels (abnormal,normal)
le1 = preprocessing.LabelEncoder()
enc_label = bin_label.apply(le1.fit_transform)
bin_data['intrusion'] = enc_label

In [ ]:
np.save("labels/le1_classes.npy",le1.classes_,allow_pickle=True)

In [22]:
# dataset with binary labels and label encoded column
bin_data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label,intrusion
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.782367,-0.280282,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,normal,1
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-1.161030,2.736852,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,1
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.938287,-0.174417,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,abnormal,0
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,normal,1
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,1.066401,-0.439078,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,normal,1


In [23]:
# one-hot-encoding attack label
bin_data = pd.get_dummies(bin_data,columns=['label'],prefix="",prefix_sep="") 
bin_data['label'] = bin_label
bin_data

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,intrusion,abnormal,normal,label
0,-0.110249,tcp,ftp_data,SF,-0.007679,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,0.069972,-0.289103,-0.639532,-0.624871,-0.224532,-0.376387,1,0,1,normal
1,-0.110249,udp,other,SF,-0.007737,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,2.367737,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
2,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal
3,-0.110249,tcp,http,SF,-0.007723,-0.002891,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.383108,0.066252,-0.572083,-0.602433,-0.387635,-0.345084,1,0,1,normal
4,-0.110249,tcp,http,SF,-0.007728,-0.004814,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,-0.110249,tcp,private,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal
125969,-0.107178,udp,private,SF,-0.007744,-0.004883,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.447834,-0.289103,-0.639532,-0.624871,-0.387635,-0.376387,1,0,1,normal
125970,-0.110249,tcp,smtp,SF,-0.007382,-0.004823,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,0.979238,-0.624871,-0.355014,-0.376387,1,0,1,normal
125971,-0.110249,tcp,klogin,S0,-0.007762,-0.004919,-0.014089,-0.089486,-0.007736,-0.095076,...,-0.480197,-0.289103,1.608759,1.618955,-0.387635,-0.376387,0,1,0,abnormal


In [24]:
# selecting attributes found by using pearson correlation coefficient
numeric_bin = bin_data[['count','srv_serror_rate','serror_rate','dst_host_serror_rate','dst_host_srv_serror_rate',
                         'logged_in','dst_host_same_srv_rate','dst_host_srv_count','same_srv_rate']]

In [25]:
# joining the selected attribute with the one-hot-encoded categorical dataframe
numeric_bin = numeric_bin.join(categorical)
# then joining encoded, one-hot-encoded, and original attack label attribute
bin_data = numeric_bin.join(bin_data[['intrusion','abnormal','normal','label']])

In [ ]:
# saving final dataset to disk
bin_data.to_csv("./datasets/bin_data.csv")
# final dataset for binary classification
bin_data

In [26]:
bin_data

,count,srv_serror_rate,serror_rate,dst_host_serror_rate,dst_host_srv_serror_rate,logged_in,dst_host_same_srv_rate,dst_host_srv_count,same_srv_rate,protocol_type_icmp,...,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,intrusion,abnormal,normal,label
0,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-0.782367,-0.818890,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
1,-0.620982,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,-1.161030,-1.035688,-1.321428,0,...,0,0,0,0,1,0,1,0,1,normal
2,0.339648,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.809857,-1.389669,0,...,1,0,0,0,0,0,0,1,0,abnormal
3,-0.690846,-0.184522,-0.189235,-0.572083,-0.602433,1.235694,1.066401,1.258754,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
4,-0.472521,-0.631929,-0.637209,-0.639532,-0.624871,1.235694,1.066401,1.258754,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125968,0.872361,1.605104,1.602664,1.608759,1.618955,-0.809262,-0.938287,-0.818890,-1.184947,0,...,1,0,0,0,0,0,0,1,0,abnormal
125969,-0.717045,-0.631929,-0.637209,-0.639532,-0.624871,-0.809262,0.977304,1.159389,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
125970,-0.725778,-0.631929,-0.637209,0.979238,-0.624871,1.235694,-0.893738,-0.773724,0.771283,0,...,0,0,0,0,1,0,1,0,1,normal
125971,0.523041,1.605104,1.602664,1.608759,1.618955,-0.809262,-1.094207,-0.972455,-1.366922,0,...,1,0,0,0,0,0,0,1,0,abnormal


---

## Salp Swarm Algorithm (SSA) as an optimizer for training a NN

In [27]:
X = bin_data.iloc[:,0:93].values # dataset excluding target attribute (encoded, one-hot-encoded,original)
Y = bin_data[['intrusion']].values # target attribute

In [28]:
# splitting the dataset 75% for training and 25% testing
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=42)


In [29]:
def setWeights(X: np.array) -> None:
 Len=0
 for i in range(0,len(mlp.layers)):
    a=mlp.layers[i].get_weights()[0].shape
    weights=X[Len:(Len+mlp.layers[i].get_weights()[0].flatten().shape[0])]
    weights=np.reshape(weights,a) 
    Len=Len+mlp.layers[i].get_weights()[0].flatten().shape[0]
    a=mlp.layers[i].get_weights()[1].shape
    bias=X[Len:(Len+mlp.layers[i].get_weights()[1].flatten().shape[0])]
    bias=np.reshape(bias,a) 
    Len=Len+mlp.layers[i].get_weights()[1].flatten().shape[0]
    mlp.layers[i].set_weights([weights,bias])


In [30]:
def Fobj():
    mlp = Sequential() # creating model
    # adding input layer and first layer with 50 neurons
    mlp.add(Dense(units=50, input_dim=X_train.shape[1], activation='relu'))
    # output layer with sigmoid activation
    mlp.add(Dense(units=1,activation='sigmoid'))
    return mlp

In [31]:
mlp = Fobj()
# defining loss function, optimizer, metrics and then compiling model
mlp.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
dim = 0
for i in range(len(mlp.get_weights())):
    dim = dim+mlp.get_weights()[i].flatten().shape[0]
print(f'dim = {dim}')
N = 20# Population size

2022-12-16 19:26:36.742131: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-16 19:26:36.742249: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-16 19:26:36.742331: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (linux-mint21-pc): /proc/driver/nvidia/version does not exist


dim = 4751


---

In [32]:
# Function: Initialize Population
def initialization(swarm_size: int, min_values: list, max_values: list) -> np.array:
  X=np.zeros((swarm_size, len(min_values)))
  for i in range(swarm_size):
    for j in range(len(min_values)):
      X[i][j] = np.random.uniform(min_values[j], max_values[j])
  return X

In [33]:
# Function: Fitness Function
def fitness(X: np.array, mlp: Sequential) -> list:
  Fit = []
  for i in range(X.shape[0]):
    setWeights(X[i, :])
    print("------------Solution ",i,"-------------")
    score = mlp.evaluate(X_train, y_train, verbose = 1)
    Fit.append(score)
    print(score[1])
  return Fit

In [34]:
# Function: Initialize Food Position
def food_position(X: np.array, Fit: list) -> np.array:
  indx = Fit.index(min(Fit))
  food = np.array(X[indx, :])
  food_fit = []
  food_fit.append(Fit[indx][0])
  food_fit.append(Fit[indx][1])
  return food, food_fit

In [35]:
# Function: Updtade Food Position by Fitness
def update_food(X: np.array, Fit: list, food: np.array, food_fit: list) -> np.array:
  indx = Fit.index(min(Fit))
  if min(Fit)[0] < food_fit[0]:
    for j in range(X.shape[1]):
      food[j] = X[indx][j]
    food_fit[0] = Fit[indx][0]
    food_fit[1] = Fit[indx][1]

  return food, food_fit

In [36]:
# Function: Updtade Position
def update_position(X: np.array, food: np.array, c1: float, min_values: list, max_values: list, mlp: Sequential) -> np.array:
  for i in range(X.shape[0]):
    if i < 1:
      for j in range(len(min_values)):
        c2 = np.random.uniform()
        c3 = np.random.uniform()
        if c3 >= 0.5:
          X[i][j] = np.clip((food[j] + c1*((max_values[j] - min_values[j])*c2 + min_values[j])), min_values[j],max_values[j])
        else:
          X[i][j] = np.clip((food[j] - c1*((max_values[j] - min_values[j])*c2 + min_values[j])), min_values[j],max_values[j])
    else:
      for j in range(len(min_values)):
        X[i][j] = np.clip(((X[i - 1,j] + X[i,j]) / 2), min_values[j],max_values[j])
  return X

In [37]:
# SSA Function
def salp_swarm_algorithm(swarm_size: int, min_values: list, max_values: list, iterations: int, mlp: Sequential) -> np.array:
  count = 0
  X = initialization(swarm_size, min_values, max_values)
  Fit = fitness(X, mlp)
  food, food_fit = food_position(X, Fit)
  print("The initialize food loss: f(x) = ", food_fit[0])
  while count <= iterations:
    c1 = 2 * math.exp(-(4 * (count / iterations))**2)
    food, food_fit = update_food(X, Fit, food, food_fit)
    print("Iteration = ", count, " f(x) = ", food_fit[0])
    X = update_position(X, food, c1, min_values, max_values, mlp)
    Fit = fitness(X, mlp)
    count += 1
  print("Iteration = ", count, " f(x) = ", food_fit[0])
  return food, food_fit

---

In [38]:
# Driven code
L_bound = -1
U_bound = 1

min_values = [L_bound for _ in range(dim)]
max_values = [U_bound for _ in range(dim)]

Food, Food_fit = salp_swarm_algorithm(N, min_values, max_values, 50, mlp)

------------Solution  0 -------------


2022-12-16 19:51:05.971746: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 35146188 exceeds 10% of free system memory.


2953/2953 [==============================] - 50s 16ms/step - loss: 1.1086 - accuracy: 0.4659
0.4659130573272705
------------Solution  1 -------------


2022-12-16 19:52:31.197653: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 35146188 exceeds 10% of free system memory.


2953/2953 [==============================] - 48s 16ms/step - loss: 1.6682 - accuracy: 0.5451
0.5450841188430786
------------Solution  2 -------------


...

2953/2953 [==============================] - 20s 7ms/step - loss: 0.2624 - accuracy: 0.8811
0.8811058402061462
------------Solution  15 -------------
2953/2953 [==============================] - 20s 7ms/step - loss: 0.2660 - accuracy: 0.8795
0.8795076012611389
------------Solution  16 -------------
2953/2953 [==============================] - 20s 7ms/step - loss: 0.2702 - accuracy: 0.8780
0.8779729008674622
------------Solution  17 -------------
2953/2953 [==============================] - 20s 7ms/step - loss: 0.2750 - accuracy: 0.8761
0.8760783076286316
------------Solution  18 -------------
2953/2953 [==============================] - 23s 8ms/step - loss: 0.2804 - accuracy: 0.8743
0.8743424415588379
------------Solution  19 -------------
2953/2953 [==============================] - 30s 10ms/step - loss: 0.2869 - accuracy: 0.8728
0.8728182911872864
Iteration =  40  f(x) =  0.2500138580799103
------------Solution  0 -------------
2953/2953 [==============================] - 23s 8ms/ste

2953/2953 [==============================] - 31s 10ms/step - loss: 0.2508 - accuracy: 0.8841
0.884122371673584
------------Solution  9 -------------
2953/2953 [==============================] - 29s 10ms/step - loss: 0.2511 - accuracy: 0.8842
0.8841647505760193
------------Solution  10 -------------
2953/2953 [==============================] - 25s 8ms/step - loss: 0.2517 - accuracy: 0.8841
0.8840906620025635
------------Solution  11 -------------
2953/2953 [==============================] - 41s 14ms/step - loss: 0.2524 - accuracy: 0.8845
0.8844716548919678
------------Solution  12 -------------
2953/2953 [==============================] - 25s 9ms/step - loss: 0.2533 - accuracy: 0.8844
0.884408175945282
------------Solution  13 -------------
2953/2953 [==============================] - 24s 8ms/step - loss: 0.2545 - accuracy: 0.8838
0.8838048577308655
------------Solution  14 -------------
2953/2953 [==============================] - 18s 6ms/step - loss: 0.2560 - accuracy: 0.8826
0.882587

2953/2953 [==============================] - 22s 7ms/step - loss: 0.2500 - accuracy: 0.8839
0.8839318752288818
------------Solution  3 -------------
2953/2953 [==============================] - 21s 7ms/step - loss: 0.2500 - accuracy: 0.8839
0.8839318752288818
------------Solution  4 -------------
2953/2953 [==============================] - 20s 7ms/step - loss: 0.2500 - accuracy: 0.8840
0.8840271234512329
------------Solution  5 -------------
2953/2953 [==============================] - 24s 8ms/step - loss: 0.2501 - accuracy: 0.8840
0.8840271234512329
------------Solution  6 -------------
2953/2953 [==============================] - 40s 14ms/step - loss: 0.2501 - accuracy: 0.8840
0.8839954137802124
------------Solution  7 -------------
2953/2953 [==============================] - 23s 8ms/step - loss: 0.2502 - accuracy: 0.8841
0.8841118216514587
------------Solution  8 -------------
2953/2953 [==============================] - 21s 7ms/step - loss: 0.2503 - accuracy: 0.8842
0.88417530059

2953/2953 [==============================] - 33s 11ms/step - loss: 0.2532 - accuracy: 0.8844
0.884418785572052
------------Solution  17 -------------
2953/2953 [==============================] - 38s 13ms/step - loss: 0.2542 - accuracy: 0.8840
0.8840482831001282
------------Solution  18 -------------
2953/2953 [==============================] - 41s 14ms/step - loss: 0.2556 - accuracy: 0.8829
0.8828840255737305
------------Solution  19 -------------
2953/2953 [==============================] - 40s 13ms/step - loss: 0.2573 - accuracy: 0.8826
0.882566511631012
Iteration =  48  f(x) =  0.24999718368053436
------------Solution  0 -------------
2953/2953 [==============================] - 28s 9ms/step - loss: 0.2500 - accuracy: 0.8839
0.8839318752288818
------------Solution  1 -------------
2953/2953 [==============================] - 30s 10ms/step - loss: 0.2500 - accuracy: 0.8839
0.8839318752288818
------------Solution  2 -------------
2953/2953 [==============================] - 36s 12ms/s

2953/2953 [==============================] - 26s 9ms/step - loss: 0.2501 - accuracy: 0.8840
0.8840059638023376
------------Solution  11 -------------
2953/2953 [==============================] - 27s 9ms/step - loss: 0.2502 - accuracy: 0.8840
0.8839636445045471
------------Solution  12 -------------
2953/2953 [==============================] - 27s 9ms/step - loss: 0.2503 - accuracy: 0.8842
0.8841753005981445
------------Solution  13 -------------
2953/2953 [==============================] - 24s 8ms/step - loss: 0.2505 - accuracy: 0.8841
0.8840906620025635
------------Solution  14 -------------
2953/2953 [==============================] - 21s 7ms/step - loss: 0.2508 - accuracy: 0.8840
0.8840482831001282
------------Solution  15 -------------
2953/2953 [==============================] - 21s 7ms/step - loss: 0.2511 - accuracy: 0.8841
0.8840800523757935
------------Solution  16 -------------
2953/2953 [==============================] - 20s 7ms/step - loss: 0.2515 - accuracy: 0.8840
0.883984

In [39]:
print('The final solution:')
print(f'loss: {Food_fit[0]}')
print(f'accurcey" {Food_fit[1]}')

The final solution:
loss: 0.24999718368053436
accurcey" 0.8839318752288818


---